In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [3]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    input_index = np.array(range(0, length, 24))
    label_index = input_index + 24* (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), pred_day, labels

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e) 


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters['watershed']

'yeong_auto_13'

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, True, 20)

In [12]:
run_num = range(len(folder))
real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)

#ddday = 31
#mmmonth = 12

#length = 1

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])
    #df, times, mmmonth, ddday = make_dataframe(data_path+folder[idx], file_names[idx], 
#                               colum_idx[idx], interpolation=interpolation_option[idx],
#                              first_file_no=i, month=mmmonth, day=ddday)

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/yeong/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['나주_2013.xlsx', '나주_2014.xlsx', '나주_2015.xlsx', '나주_2016.xlsx', '나주_2017.xlsx', '나주_2018.xlsx', '나주_2019.xlsx', '나주_2020.xlsx'], ['용봉_2013.xlsx', '용봉_2014.xlsx', '용봉_2015.xlsx', '용봉_2016.xlsx', '용봉_2017.xlsx', '용봉_2018.xlsx', '용봉_2019.xlsx', '용봉_2020.xlsx'], ['우치_2013.xlsx', '우치_2014.xlsx', '우치_2015.xlsx', '우치_2016.xlsx', '우치_2017.xlsx', '우치_2018.xlsx', '우치_2019.xlsx', '우치_2020.xlsx']]
data/yeong/자동/나주_2013.xlsx
data/yeong/자동/나주_2014.xlsx
data/yeong/자동/나주_2015.xlsx
data/yeong/자동/나주_2016.xlsx
data/yeong/자동/나주_2017.xlsx
data/yeong/자동/나주_2018.xlsx
data/yeong/자동/나주_2019.xlsx
data/yeong/자동/나주_2020.xlsx
You must check this make_timeseries : time length
time range in files :  2013-01-01 00:00  ~  2020-12-31 23:00
data/yeong/자동/용봉_2013.xlsx
data/yeong/자동/용봉_2014.xlsx
data/yeong/자동/용봉_2015.xlsx
data/yeong/자동/용봉_2016.xlsx
data/yeong/자동/용봉_2017.xlsx
data/yeong/자동/용봉_2018.xlsx
dat

1/1 [==============================] - 0s 20ms/step - gen_loss: 12.9173 - disc_loss: 0.3357 - rmse: 0.4657 - val_loss: 0.5088
Epoch 49/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 14.7465 - disc_loss: 0.3571 - rmse: 0.4729 - val_loss: 0.4861
Epoch 50/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 12.0819 - disc_loss: 0.3389 - rmse: 0.4389 - val_loss: 0.4580
Epoch 51/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 12.5593 - disc_loss: 0.3341 - rmse: 0.4579 - val_loss: 0.4666
Epoch 52/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.8361 - disc_loss: 0.3474 - rmse: 0.4264 - val_loss: 0.4222
Epoch 53/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.5236 - disc_loss: 0.3319 - rmse: 0.3944 - val_loss: 0.3905
Epoch 54/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.7499 - disc_loss: 0.3208 - rmse: 0.4061 - val_loss: 0.4545
Epoch 55/2000
1/1 [===============

Epoch 107/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.5428 - disc_loss: 0.2659 - rmse: 0.3501 - val_loss: 0.4139
Epoch 108/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.6159 - disc_loss: 0.2511 - rmse: 0.3813 - val_loss: 0.3999
Epoch 109/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.4583 - disc_loss: 0.2747 - rmse: 0.3526 - val_loss: 0.3958
Epoch 110/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.5769 - disc_loss: 0.2754 - rmse: 0.3547 - val_loss: 0.3445
Epoch 111/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.8914 - disc_loss: 0.2556 - rmse: 0.3606 - val_loss: 0.3417
Epoch 112/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.7622 - disc_loss: 0.2637 - rmse: 0.4116 - val_loss: 0.3600
Epoch 113/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.2376 - disc_loss: 0.2445 - rmse: 0.3239 - val_loss: 0.3611
Epoch 114/2000
1/1 [

1/1 [==============================] - 0s 20ms/step - gen_loss: 7.5347 - disc_loss: 0.2452 - rmse: 0.3909 - val_loss: 0.3443
Epoch 166/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.3416 - disc_loss: 0.2338 - rmse: 0.3214 - val_loss: 0.3254
Epoch 167/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.4533 - disc_loss: 0.2495 - rmse: 0.3362 - val_loss: 0.3577
Epoch 168/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.3835 - disc_loss: 0.2237 - rmse: 0.3150 - val_loss: 0.3086
Epoch 169/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.3734 - disc_loss: 0.2429 - rmse: 0.3216 - val_loss: 0.2809
Epoch 170/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.8708 - disc_loss: 0.2289 - rmse: 0.3687 - val_loss: 0.3320
Epoch 171/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.5576 - disc_loss: 0.2241 - rmse: 0.3474 - val_loss: 0.3359
Epoch 172/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.2883 - disc_loss: 0.2340 - rmse: 0.3076 - val_loss: 0.3178
Epoch 224/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.3165 - disc_loss: 0.2282 - rmse: 0.3195 - val_loss: 0.2992
Epoch 225/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.0243 - disc_loss: 0.2147 - rmse: 0.3372 - val_loss: 0.3221
Epoch 226/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.3701 - disc_loss: 0.2315 - rmse: 0.3742 - val_loss: 0.3580
Epoch 227/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.9656 - disc_loss: 0.2215 - rmse: 0.3168 - val_loss: 0.3177
Epoch 228/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.1847 - disc_loss: 0.2084 - rmse: 0.3169 - val_loss: 0.3265
Epoch 229/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 6.8457 - disc_loss: 0.2234 - rmse: 0.3716 - val_loss: 0.3073
Epoch 230/2000
1/1 [===============

...

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4388 - disc_loss: 0.1610 - rmse: 0.2771 - val_loss: 0.3005
Epoch 1442/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2755 - disc_loss: 0.1598 - rmse: 0.2614 - val_loss: 0.2886
Epoch 1443/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0764 - disc_loss: 0.1577 - rmse: 0.2381 - val_loss: 0.3376
Epoch 1444/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3217 - disc_loss: 0.1576 - rmse: 0.3054 - val_loss: 0.3235
Epoch 1445/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6015 - disc_loss: 0.1492 - rmse: 0.2726 - val_loss: 0.2960
Epoch 1446/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3443 - disc_loss: 0.1537 - rmse: 0.2610 - val_loss: 0.2371
Epoch 1447/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7730 - disc_loss: 0.1555 - rmse: 0.2744 - val_loss: 0.3108
Epoch 1448/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 4.1536 - disc_loss: 0.1423 - rmse: 0.3170 - val_loss: 0.3010
Epoch 1500/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7932 - disc_loss: 0.1551 - rmse: 0.2774 - val_loss: 0.2472
Epoch 1501/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4964 - disc_loss: 0.1555 - rmse: 0.2451 - val_loss: 0.2935
Epoch 1502/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.0015 - disc_loss: 0.1587 - rmse: 0.2715 - val_loss: 0.3097
Epoch 1503/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.9839 - disc_loss: 0.1658 - rmse: 0.2732 - val_loss: 0.2956
Epoch 1504/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.1837 - disc_loss: 0.1583 - rmse: 0.2837 - val_loss: 0.3780
Epoch 1505/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.9917 - disc_loss: 0.1677 - rmse: 0.3213 - val_loss: 0.2893
Epoch 1506/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5061 - disc_loss: 0.1540 - rmse: 0.2706 - val_loss: 0.3073
Epoch 1558/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3634 - disc_loss: 0.1512 - rmse: 0.2430 - val_loss: 0.3120
Epoch 1559/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4925 - disc_loss: 0.1655 - rmse: 0.2888 - val_loss: 0.3028
Epoch 1560/2000
1/1 [==============================] - 0s 409ms/step - gen_loss: 3.0778 - disc_loss: 0.1612 - rmse: 0.2487 - val_loss: 0.2573
Epoch 1561/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7285 - disc_loss: 0.1616 - rmse: 0.2879 - val_loss: 0.2793
Epoch 1562/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.2055 - disc_loss: 0.1763 - rmse: 0.3027 - val_loss: 0.2997
Epoch 1563/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7473 - disc_loss: 0.1507 - rmse: 0.2779 - val_loss: 0.2732
Epoch 1564/2000
1/1 [=======

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5315 - disc_loss: 0.1556 - rmse: 0.2876 - val_loss: 0.2458
Epoch 1616/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0822 - disc_loss: 0.1652 - rmse: 0.2617 - val_loss: 0.3039
Epoch 1617/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7619 - disc_loss: 0.1706 - rmse: 0.2966 - val_loss: 0.2669
Epoch 1618/2000
1/1 [==============================] - 0s 22ms/step - gen_loss: 3.1962 - disc_loss: 0.1519 - rmse: 0.2500 - val_loss: 0.3171
Epoch 1619/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5346 - disc_loss: 0.1584 - rmse: 0.3098 - val_loss: 0.2895
Epoch 1620/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.8911 - disc_loss: 0.1669 - rmse: 0.2775 - val_loss: 0.3477
Epoch 1621/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.9424 - disc_loss: 0.1645 - rmse: 0.2928 - val_loss: 0.2939
Epoch 1622/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 4.0397 - disc_loss: 0.1604 - rmse: 0.3495 - val_loss: 0.2752
Epoch 1674/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3831 - disc_loss: 0.1632 - rmse: 0.2570 - val_loss: 0.3147
Epoch 1675/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2331 - disc_loss: 0.1620 - rmse: 0.2663 - val_loss: 0.3200
Epoch 1676/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4363 - disc_loss: 0.1535 - rmse: 0.3053 - val_loss: 0.2880
Epoch 1677/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4418 - disc_loss: 0.1543 - rmse: 0.2871 - val_loss: 0.2701
Epoch 1678/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4882 - disc_loss: 0.1474 - rmse: 0.3019 - val_loss: 0.2521
Epoch 1679/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8596 - disc_loss: 0.1567 - rmse: 0.3236 - val_loss: 0.3000
MissData :  save/  miss :  (1

In [14]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [15]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  6
out_num_features :  1


In [45]:
print(real_df_all.shape)
rnn_target_column = 'tn'
print(rnn_target_column)

#df_length = real_df_all.shape[0] - 3000
df_length = real_df_all.shape[0]

fake_df = real_df_all.iloc[:df_length,:]

(70128, 36)
tn


In [46]:
#train_df, val_df, test_df, test_df2 = dataset_slice(fake_df, 0.8, 0.1, 0.1)
#teg_check = 'train=8, val=1, test=1'

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

#val_df, test_df,train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'val=1, test=1, train=8'

#test_df, val_df, train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'test=1, val=1, train=8'

#val_df, train_df, test_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'val=1, train=8, test=1'

#test_df, train_df, val_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'test=1, train=8, val=1'

train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, test=1, val=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (49089, 36) val_df.shape :  (14026, 36) test_df.shape: (7013, 36)


In [47]:
#fake_df.shape[0]/8760

In [48]:
plt.figure()
plt.plot(real_df_all.iloc[:, out_features[0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [49]:
plt.figure()
plt.plot(val_df.iloc[:, out_features[0]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [50]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [51]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

target_col_idx :  5
out_num_features :  1


In [52]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}

In [53]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

model_path = "save/" + watershed + "models/" + file_parameters['watershed'] + '/'+  pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    #if not os.path.exists('save/' + watershed):
    #    os.makedirs('save/' + watershed)
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/yeong/models/yeong_auto_13/nitrogen/


In [54]:
#"save/" + watershed + "models/" + file_parameters['watershed'] + '/' + pa[indices[target_col_idx]]
#model_path

In [55]:
rnn_epochs

20

## 모델 학습

In [ ]:
gru_model = model_gru(
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=50,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt")

Epoch 1/50
10/10 [==============================] - ETA: 0s - loss: 0.3482 - mean_absolute_error: 0.4321 - nse: 0.2886
Epoch 00001: val_nse improved from -inf to 0.59714, saving model to save/best_model.h5
10/10 [==============================] - 10s 989ms/step - loss: 0.3482 - mean_absolute_error: 0.4321 - nse: 0.2886 - val_loss: 0.1892 - val_mean_absolute_error: 0.3439 - val_nse: 0.5971
Epoch 2/50
10/10 [==============================] - ETA: 0s - loss: 0.2010 - mean_absolute_error: 0.3279 - nse: 0.6234
Epoch 00002: val_nse improved from 0.59714 to 0.59815, saving model to save/best_model.h5
10/10 [==============================] - 10s 970ms/step - loss: 0.2010 - mean_absolute_error: 0.3279 - nse: 0.6234 - val_loss: 0.1875 - val_mean_absolute_error: 0.3353 - val_nse: 0.5982
Epoch 3/50
10/10 [==============================] - ETA: 0s - loss: 0.1324 - mean_absolute_error: 0.2682 - nse: 0.7472
Epoch 00003: val_nse improved from 0.59815 to 0.71450, saving model to save/best_model.h5
10/1

Epoch 22/50
10/10 [==============================] - ETA: 0s - loss: 0.0847 - mean_absolute_error: 0.2028 - nse: 0.8454
Epoch 00022: val_nse did not improve from 0.78968
10/10 [==============================] - 10s 972ms/step - loss: 0.0847 - mean_absolute_error: 0.2028 - nse: 0.8454 - val_loss: 0.1067 - val_mean_absolute_error: 0.2418 - val_nse: 0.7723
Epoch 23/50
10/10 [==============================] - ETA: 0s - loss: 0.0853 - mean_absolute_error: 0.2037 - nse: 0.8284
Epoch 00023: val_nse did not improve from 0.78968
10/10 [==============================] - 10s 965ms/step - loss: 0.0853 - mean_absolute_error: 0.2037 - nse: 0.8284 - val_loss: 0.1126 - val_mean_absolute_error: 0.2467 - val_nse: 0.7591
Epoch 24/50
10/10 [==============================] - ETA: 0s - loss: 0.0776 - mean_absolute_error: 0.1999 - nse: 0.8550
Epoch 00024: val_nse did not improve from 0.78968
10/10 [==============================] - 10s 985ms/step - loss: 0.0776 - mean_absolute_error: 0.1999 - nse: 0.8550 - v

10/10 [==============================] - ETA: 0s - loss: 0.0561 - mean_absolute_error: 0.1719 - nse: 0.8869
Epoch 00045: val_nse did not improve from 0.78968
10/10 [==============================] - 10s 983ms/step - loss: 0.0561 - mean_absolute_error: 0.1719 - nse: 0.8869 - val_loss: 0.1425 - val_mean_absolute_error: 0.2747 - val_nse: 0.6965
Epoch 46/50
10/10 [==============================] - ETA: 0s - loss: 0.0546 - mean_absolute_error: 0.1697 - nse: 0.8860
Epoch 00046: val_nse did not improve from 0.78968
10/10 [==============================] - 10s 971ms/step - loss: 0.0546 - mean_absolute_error: 0.1697 - nse: 0.8860 - val_loss: 0.1578 - val_mean_absolute_error: 0.2888 - val_nse: 0.6642
Epoch 47/50
10/10 [==============================] - ETA: 0s - loss: 0.0514 - mean_absolute_error: 0.1653 - nse: 0.8904
Epoch 00047: val_nse did not improve from 0.78968
10/10 [==============================] - 10s 971ms/step - loss: 0.0514 - mean_absolute_error: 0.1653 - nse: 0.8904 - val_loss: 0.1

## core / window.py / 

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
#train_df.shape, val_df.shape, test_df.shape

In [ ]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvbnfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************       (*_*)  
#                   SSSSSSSS               ******            ******              *****    *****            ~( (>
#        SSSS         S  SSS               ******            ******              *****    *****            / (          
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=real_df_all, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
from core.models import GRUModel, compile

In [ ]:
model112 = GRUModel(rnn_out_steps, out_num_features)
model112.load_weights(model_path+"gru.ckpt")
compile(model112)


In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    #model=model112,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    model=model112,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
##.evaluate(multi_window.train)
#model112.evaluate(multi_window.val.repeat(-1), steps=100)

In [ ]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    #print(df.shape)
    #print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    

    predictions = model(inputs).numpy()
    
    print("input : ", inputs.shape, "labels : ", labels.shape, "predictions : ", predictions.shape)
    #print(predictions.shape)
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    print("                label_day.shape : ",label_day.shape, "pred_day.shape : ", pred_day.shape)
    
    #plt.figure()
    #input_index = np.array(range(0, length, 24))
    #label_index = input_index + 24* (7 + predict_day)
    #plt.plot(input_index, inputs_day[:, 0, :], label='input')
    #plt.plot(label_index, label_day[:, predict_day, :], label='label')
    #plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    #plt.legend()
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), inputs_day, label_day, pred_day
    

In [ ]:

x = np.arange(len(val_nse))
width = 0.35
plt.figure()
plt.title(watershed + '  ['+start_year+','+end_year+']  ' + rnn_target_column)
plt.bar(x, val_pbias.values(), 0.3, label='PBIAS' )
plt.bar(x + width, val_nse.values(), 0.3, label='NSE')
plt.xticks(x,val_nse.keys(), rotation=0)
_ = plt.legend()
plt.show()
